In [1]:
import numpy as np
import pandas as pd

import os

import math
from tqdm import tqdm

In [4]:
raw_data = pd.read_csv('/opt/ml/input/data/FE_total_data.csv', parse_dates=["Timestamp"])
raw_data.sort_values(by=["userID", "Timestamp"], inplace=True)
df = raw_data.copy()

In [5]:
df["grade"] = df["testId"].apply(lambda x: int(x[1:4]) // 10)    
df["problem_number"] = df["assessmentItemID"].apply(lambda x: int(x[-3:]))

gdf = df[["userID", "testId", "problem_number", "grade", "Timestamp"]].sort_values(
    by=["userID", "grade", "Timestamp"]
)
gdf["buserID"] = gdf["userID"] != gdf["userID"].shift(1)
gdf["bgrade"] = gdf["grade"] != gdf["grade"].shift(1)
gdf["first"] = gdf[["buserID", "bgrade"]].any(axis=1).apply(lambda x: 1 - int(x))
gdf["RepeatedTime"] = gdf["Timestamp"].diff().fillna(pd.Timedelta(seconds=0))
gdf["RepeatedTime"] = (
    gdf["RepeatedTime"].apply(lambda x: x.total_seconds()) * gdf["first"]
)

df["RepeatedTime"] = gdf["RepeatedTime"].apply(lambda x: math.log(x + 1))


In [7]:
df.to_csv('/opt/ml/input/data/FE/RepeatedTime.csv', columns=["userID", "assessmentItemID", "testId", "answerCode", "Timestamp", "KnowledgeTag", "dataset", "RepeatedTime"], index=False)